<a href="https://colab.research.google.com/github/Ma7moudYasser/Sentiment_analysis_using_cnn_bilstm/blob/main/CNN_using_BilSTM_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mounting data to drive

In [1]:

from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pandas as pd

directory  = pd.read_excel('/content/drive/MyDrive/cnn-bilstm/capstone_airline_reviews3.xlsx')
directory.head()



,airline,overall,author,review_date,customer_review,aircraft,traveller_type,cabin,route,date_flown,seat_comfort,cabin_service,food_bev,entertainment,ground_service,value_for_money,recommended
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Turkish Airlines,7.0,Christopher Hackley,8th May 2019,âœ… Trip Verified | London to Izmir via Istanb...,NaN,Business,Economy Class,London to Izmir via Istanbul,2019-05-01 00:00:00,4.0,5.0,4.0,4.0,2.0,4.0,yes
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Turkish Airlines,2.0,Adriana Pisoi,7th May 2019,âœ… Trip Verified | Istanbul to Bucharest. We ...,NaN,Family Leisure,Economy Class,Istanbul to Bucharest,2019-05-01 00:00:00,4.0,1.0,1.0,1.0,1.0,1.0,no
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Removing Empty rows

In [3]:
df = directory.dropna(subset=['customer_review', 'recommended'])

counting_nan_values = df.isna().sum()


print(counting_nan_values)

airline                0
overall             1910
author                 0
review_date            0
customer_review        0
aircraft           44723
traveller_type     24686
cabin               1138
route              24715
date_flown         24807
seat_comfort        3759
cabin_service       3725
food_bev           11832
entertainment      20247
ground_service     25082
value_for_money      465
recommended            0
dtype: int64


In [4]:
print(df['customer_review'])

1         âœ… Trip Verified | London to Izmir via Istanb...
3         âœ… Trip Verified | Istanbul to Bucharest. We ...
5         âœ… Trip Verified | Rome to Prishtina via Ista...
7         âœ… Trip Verified | Flew on Turkish Airlines I...
9         âœ… Trip Verified | Mumbai to Dublin via Istan...
                                ...                        
131886    Two domestic flights between Kiev and Lviv las...
131888    I had a good flight from London to Kiev with U...
131890    Kiev - London (Gatwick) in business class (in ...
131892    Several flights - KBP to AMS (3 times one way)...
131894    KBP-AMS with UIA. Although it was a relatively...
Name: customer_review, Length: 64440, dtype: object


## Applying glove embedding for the customer_review column

In [5]:
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from gensim.models import KeyedVectors


nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
def preprocess_text(text):
  tokens = word_tokenize(text.lower())
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [word for word in tokens if word not in stop_words]
  return tokens


df['customer_review'] = df['customer_review'].apply(preprocess_text)



<ipython-input-6-eea4d2430585>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['customer_review'] = df['customer_review'].apply(preprocess_text)


In [7]:
!pip install nlu

In [8]:
!pip install pyspark==3.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.2/204.2 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 25.2 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612223 sha256=57a0b27e6e05598d04c9607faf827052653cfef2b63661b20b7e7c4cfc2761ed
  Stored in directory: /root/.cache/pip/wheels/19/b0/c8/6cb894117070e130fc44352c2a13f15b6c27e440d04a84fb48
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [9]:
import nlu
glove_embedding = nlu.load('glove')


glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]


## Adding glove embedding

In [11]:
glove_file = '/content/drive/MyDrive/cnn-bilstm/glove.6B.50d.txt'


file_path = '/content/drive/MyDrive/cnn-bilstm/glove.6B.50d.txt'


word_vectors = KeyedVectors.load_word2vec_format(file_path, binary = False, encoding = 'utf8', no_header = True)

def compute_sentence_embedding(tokens):
    embeddings = []
    for token in tokens:
        if token in word_vectors:
            embeddings.append(word_vectors[token])
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(word_vectors.vector_size)





df['embedding'] = df['customer_review'].apply(compute_sentence_embedding)


## Checking model dimension

In [12]:
print(df['embedding'].shape)  # Check the shape of embeddings
print(df['recommended'].shape)  # Check the shape of labels (should be the same)


(64440,)
(64440,)


In [13]:
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Bidirectional, LSTM, GlobalMaxPooling1D
from tensorflow.keras.preprocessing.sequence import pad_sequences




label_mapping = {'yes': 1, 'no': 0}
df['recommended'] = df['recommended'].map(label_mapping)
print(df['recommended'].shape)


X = np.vstack(df['embedding'])
X = np.expand_dims(X, axis=1)  # Reshape the input data
print(X.shape)
y = df['recommended']
print(y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)


max_len = 50  # Adjust this based on the longest sentence length in your data
X_train_padded = pad_sequences(X_train, maxlen=max_len, padding='post')
X_test_padded = pad_sequences(X_test, maxlen=max_len, padding='post')


model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True, input_shape=(X_train.shape[1],))))


model.add(GlobalMaxPooling1D())

model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))

model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

model.fit(X_train, y_train, epochs = 10, batch_size = 32, validation_data= (X_test, y_test))

(64440,)
(64440, 1, 50)
(64440,)
Epoch 1/10
1611/1611 [==============================] - 20s 8ms/step - loss: 0.4014 - accuracy: 0.8225 - val_loss: 0.3833 - val_accuracy: 0.8261
Epoch 2/10
1611/1611 [==============================] - 12s 7ms/step - loss: 0.3804 - accuracy: 0.8330 - val_loss: 0.3939 - val_accuracy: 0.8222
Epoch 3/10
1611/1611 [==============================] - 10s 6ms/step - loss: 0.3784 - accuracy: 0.8341 - val_loss: 0.3871 - val_accuracy: 0.8313
Epoch 4/10
1611/1611 [==============================] - 12s 8ms/step - loss: 0.3730 - accuracy: 0.8370 - val_loss: 0.3714 - val_accuracy: 0.8322
Epoch 5/10
1611/1611 [==============================] - 12s 7ms/step - loss: 0.3690 - accuracy: 0.8394 - val_loss: 0.3733 - val_accuracy: 0.8356
Epoch 6/10
1611/1611 [==============================] - 13s 8ms/step - loss: 0.3660 - accuracy: 0.8415 - val_loss: 0.3663 - val_accuracy: 0.8354
Epoch 7/10
1611/1611 [==============================] - 12s 8ms/step - loss: 0.3628 - accuracy: 0

## Saving and testing the model

In [15]:

from tensorflow.keras.callbacks import ModelCheckpoint
model_checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/MyDrive/cnn-bilstm/best_model.h5',
    save_best_only=True,
    monitor='val_accuracy'
)
model.fit(X_train, y_train, epochs=10, batch_size=32,
          validation_data=(X_test_padded, y_test), callbacks=[model_checkpoint_callback])


Epoch 1/10
1611/1611 [==============================] - 15s 9ms/step - loss: 0.3535 - accuracy: 0.8464 - val_loss: 0.7186 - val_accuracy: 0.4657
Epoch 2/10
1611/1611 [==============================] - 11s 7ms/step - loss: 0.3522 - accuracy: 0.8471 - val_loss: 0.7514 - val_accuracy: 0.4657
Epoch 3/10
1611/1611 [==============================] - 11s 7ms/step - loss: 0.3490 - accuracy: 0.8488 - val_loss: 0.7172 - val_accuracy: 0.4657
Epoch 4/10
1611/1611 [==============================] - 12s 7ms/step - loss: 0.3469 - accuracy: 0.8499 - val_loss: 0.7619 - val_accuracy: 0.4657
Epoch 5/10
1611/1611 [==============================] - 12s 7ms/step - loss: 0.3448 - accuracy: 0.8501 - val_loss: 0.7677 - val_accuracy: 0.4657
Epoch 6/10
1611/1611 [==============================] - 11s 7ms/step - loss: 0.3422 - accuracy: 0.8518 - val_loss: 0.7367 - val_accuracy: 0.4657
Epoch 7/10
1611/1611 [==============================] - 11s 7ms/step - loss: 0.3396 - accuracy: 0.8534 - val_loss: 0.8515 - val_ac

In [16]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

403/403 [==============================] - 2s 5ms/step - loss: 0.3652 - accuracy: 0.8393
Test Loss: 0.3652029037475586
Test Accuracy: 0.8393079042434692
